In [8]:
import pandas as pd

# Load input data
advertiser_data = {
    'Month': ['2025-01', '2025-02', '2025-03', '2025-04', '2025-05', '2025-06',
              '2025-07', '2025-08', '2025-09', '2025-10', '2025-11', '2025-12'],
    'Advertisers': [1143, 1500, 1720, 2037, 1230, 2832, 1600, 1000, 809, 1362, 1432, 1800],
    'Avg_Budget': [81099, 91100, 71101, 77102, 89403, 87604, 100050, 115390, 95283, 83293, 102689, 98244]
}

# Global parameters
initial_agents = 652
annual_salary = 77721
monthly_salary = annual_salary / 12
firing_cost_percentage = 0.4
max_advertisers_per_agent = 10
support_duration = 2
ramp_up_time = 1
expected_uplift = 0.125

# Initialize tracking variables
months = advertiser_data['Month']
n_months = len(months)
A = [initial_agents] + [0] * (n_months - 1)  # Available agents
N = [0] * n_months  # New hires
F = [0] * n_months  # Fired agents
S = [0] * n_months  # Advertisers assigned
W = [0] * n_months  # Waiting pool
costs = [0] * n_months
revenue_uplift = [0] * n_months

# Simulate month-by-month staffing
for m in range(n_months):
    # Calculate required agents for current + previous month's advertisers
    if m == 0:
        required_agents = advertiser_data['Advertisers'][m] / max_advertisers_per_agent
    else:
        required_agents = (advertiser_data['Advertisers'][m] + advertiser_data['Advertisers'][m-1]) / max_advertisers_per_agent

    # Calculate staffing gap
    gap = required_agents - A[m]

    # Determine hiring/firing (within constraints)
    if gap > 0:  # Need more agents
        max_hires = 0.2 * A[m]
        N[m] = min(gap, max_hires)
        F[m] = 0
    else:  # Need fewer agents
        max_fires = 0.25 * A[m]
        F[m] = min(-gap, max_fires)
        N[m] = 0

    # Update next month's available agents (with ramp-up delay)
    if m < n_months - 1:
        A[m+1] = A[m] + N[m] - F[m]

    # Assign advertisers (S_m <= A_m * 10)
    S[m] = min(advertiser_data['Advertisers'][m] + W[m-1 if m>0 else 0],
                A[m] * max_advertisers_per_agent)
    W[m] = max(0, (advertiser_data['Advertisers'][m] + (W[m-1] if m>0 else 0) - S[m]))

    # Calculate costs and revenue
    salary_cost = A[m] * monthly_salary
    firing_cost = F[m] * annual_salary * firing_cost_percentage
    uplift = S[m] * advertiser_data['Avg_Budget'][m] * expected_uplift

    costs[m] = salary_cost + firing_cost - uplift
    revenue_uplift[m] = uplift

# Compile results into DataFrame
results = pd.DataFrame({
    'Month': months,
    'Available Agents': [round(a) for a in A],
    'New Hires': N,
    'Fired': F,
    'Advertisers Assigned': S,
    'Total Cost': [round(c, 2) for c in costs],
    'Revenue Uplift': [round(u, 2) for u in revenue_uplift]
})

# Print key metrics
total_cost = sum(results['Total Cost'])
total_uplift = sum(results['Revenue Uplift'])
net_profit = total_uplift - total_cost

print("Optimized Monthly Staffing Plan:\n")
print(results.to_string(index=False))
print(f"\nTotal Annual Cost: ${total_cost:,.2f}")
print(f"Total Revenue Uplift: ${total_uplift:,.2f}")
print(f"Net Profit: ${net_profit:,.2f}")

Optimized Monthly Staffing Plan:

  Month  Available Agents  New Hires  Fired  Advertisers Assigned   Total Cost  Revenue Uplift
2025-01               652      0.000 163.00                  1143  -2296769.43     11587019.62
2025-02               489      0.000 122.25                  1500 -10113562.35     17081250.00
2025-03               367      0.000  44.75                  1720 -11520161.04     15286715.00
2025-04               322     53.700   0.00                  2037 -17546583.25     19632096.75
2025-05               376      0.000  49.00                  1230  -9789064.68     13745711.25
2025-06               327     65.340   0.00                  2832 -28895861.77     31011816.00
2025-07               392     51.160   0.00                  1600 -17470854.93     20010000.00
2025-08               443      0.000 110.80                  1000  -8108659.68     14423750.00
2025-09               332      0.000  83.10                   809  -4899175.63      9635493.38
2025-10         